# 🛠️ Setup – Run once

In [7]:
# @title Code download
# @markdown This block initialises the notebook.
!rm -r sample_data
!mkdir data
!mkdir export
!mkdir plots
!wget -O wikilit.zip "https://userpage.fu-berlin.de/fu7182rx/wikilit.zip"
!unzip -o wikilit.zip
!rm wikilit.zip

# Git method
# !git clone https://github.com/v-ji/wiki-literature
# !mv wiki-literature/* .
# !rm -r wiki-literature


rm: cannot remove 'sample_data': No such file or directory
mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘export’: File exists
mkdir: cannot create directory ‘plots’: File exists
Cloning into 'wiki-literature'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 10 (delta 1), pack-reused 0
Receiving objects: 100% (10/10), 20.72 KiB | 2.30 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [8]:
# @title Package installation
# @markdown This block installs the required packages.
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.0/701.0 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.3/154.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
  Attempting uninstall: polars
    Found existing installation: polars 0.17.3
    Uninstalling polars-0.17.3:
      Successfully uninstalled polars-0.17.3
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully unin

In [9]:
# @title Package imports
# @markdown This block imports the necessary packages.
from wikilit import wikilit
import polars as pl
import plotly.express as px
# Colab files handling
from google.colab import files
import os

# 📚 Selection Method

This notebook supports multiple selection methods that determine the way the list of articles is compiled.

__Note:__ When parsing page names, formats with spaces (`Charlotte Brontë`) and underscores (`Charlotte_Brontë`) are both supported.

## Article’s language links (`langlinks`)
This method compiles all interlanguage links of a certain article, i.e. versions of the article on other editions of the site. The `selection` should be an article title.

Examples: `Charlotte Brontë`, `Virginia Wolff`, `César Aira`, `Herta Müller`, `Elfriede Jelinek`, `Christa Wolf`, `Nikos Kazantzakis`.

In [12]:
selection_method = "langlinks"
article_title = "Elfriede Jelinek" # @param ["Charlotte Brontë", "Virginia Wolff", "César Aira", "Herta Müller", "Elfriede Jelinek", "Christa Wolf", "Nikos Kazantzakis"] {allow-input: true}
selection = article_title
site = "wikipedia" # @param ["wikipedia"] {allow-input: true}
lang = "en" # @param {type:"string"}

## Articles of a category (`category`)
This method selects all articles in a MediaWiki category and its subcategories. The `selection` should be a category title.

Examples: `19th-century English women writers`, `Chinese women novelists`, `North Korean novelists`.

In [10]:
selection_method = "category"
category_title = "Brazilian LGBT novelists" # @param ["19th-century English women writers", "North Korean novelists", "Chinese women novelists", "South Korean women novelists", "Brazilian LGBT novelists"] {allow-input: true}
selection = category_title
site = "wikipedia" # @param ["wikipedia"] {allow-input: true}
lang = "en" # @param {type:"string"}

## Articles from a file (`file`)
This method reads article titles from a tab-separated values (TSV) file you supply.

The resulting dataframe retains all columns of the supplied file and includes the page stats as additional columns. This allows you to enrich existing datasets.

This method is especially useful when combined with output from the [Wikidata Query Service](https://query.wikidata.org/).

`article_title_column` determines which column will be used as the article title.

In [4]:
selection_method = "file"
article_title_column = "desc"  # @param {type:"string"}
site = "wikipedia"  # @param ["wikipedia"] {allow-input: true}
lang = "en" # @param ["en", "de", "fr", "es", "ja", "ru", "pt", "zh", "it", "ar"] {allow-input: true}

# Upload file
uploaded = files.upload()
file_list = list(uploaded.keys())

# Check file
if len(file_list) > 1:
    for file in file_list:
        os.remove(file)
    raise ValueError("Please only upload one file.")

filename = file_list[0]
if not filename.endswith(".tsv"):
    os.remove(filename)
    raise ValueError("File must be TSV.")

# Move file to data/ directory
dest = "data/input.tsv"
os.rename(filename, dest)
print(f"File saved as {dest}")
selection = dest

Saving test.tsv to test.tsv
File saved as data/input.tsv


# 🛰️ Data Retrieval

This section retrieves the data from the pages you selected above.

## Columns
- `length`: Length of the article in bytes
- `n_contributors`: Number of contributors
- `n_revisions`: Number of revisions
- `n_extlinks`: Number of external links
- `n_langlinks`: Number of interlanguage links
- `n_links`: Number of internal links
- `n_linkshere`: Number of pages linking to this page
- `n_categories`: Number of categories the page is in
- `pageviews_60d`: Number of pageviews in the last 60 days
- `first_revision`: Date of the first revision (i.e. creation date)

### Metadata columns
- `selection_method`: The method used to select the articles
- `selection`: The selection used to select the articles
- `selection_case`: The name of the case within the selection

The data is stored in a [Polars](https://pola.rs/) dataframe, which is similar to a Pandas dataframe.

In [11]:
# @title Fetch
print(
    f"Retrieving data using selection_method='{selection_method}' and selection='{selection}' from {lang}.{site}"
)

if "article_title_column" not in locals():
    article_title_column = None

data = wikilit(
  selection_method=selection_method,
  selection=selection,
  lang=lang,
  site=site,
  article_title_column=article_title_column
)

print(data)
data.write_csv("data/output.tsv", separator="\t")
print("Wrote data to data/output.tsv")


Retrieving data using selection_method='category' and selection='Brazilian LGBT novelists' from en.wikipedia


Getting page stats: 100%|██████████| 10/10 [00:38<00:00,  3.84s/it, João Silvério Trevisan]

shape: (10, 15)
┌────────────┬────────────┬────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ title      ┆ url        ┆ length ┆ n_contrib ┆ … ┆ first_rev ┆ selection ┆ selection ┆ selection │
│ ---        ┆ ---        ┆ ---    ┆ utors     ┆   ┆ ision     ┆ _method   ┆ ---       ┆ _case     │
│ str        ┆ str        ┆ i64    ┆ ---       ┆   ┆ ---       ┆ ---       ┆ str       ┆ ---       │
│            ┆            ┆        ┆ i64       ┆   ┆ datetime[ ┆ str       ┆           ┆ str       │
│            ┆            ┆        ┆           ┆   ┆ μs]       ┆           ┆           ┆           │
╞════════════╪════════════╪════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ Lúcio      ┆ https://en ┆ 7806   ┆ 44        ┆ … ┆ 2008-05-0 ┆ category  ┆ Brazilian ┆ Lúcio     │
│ Cardoso    ┆ .wikipedia ┆        ┆           ┆   ┆ 5         ┆           ┆ LGBT      ┆ Cardoso   │
│            ┆ .org/wiki/ ┆        ┆           ┆   ┆ 12:45:00  ┆           

In [10]:
# @title Data export
format = "TSV"  # @param ["TSV", "Excel"]
filename = "output"  # @param {type:"string"}

export_dir = "export/"

output_file = "" + export_dir + filename

if format == "TSV":
    output_file += ".tsv"
    data.write_csv(output_file, separator="\t")
elif format == "Excel":
    output_file += ".xlsx"
    data.write_excel(output_file)

files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 📊 Plotting

This section allows you to plot your data onto a graph.


In [12]:
# @title Bar plot

template = "plotly_white"
yaxis_var = "length" # @param ["length", "n_contributors", "n_revisions", "n_extlinks", "n_langlinks", "n_links", "n_linkshere", "n_categories", "pageviews_60d", "first_revision"] {allow-input: true}
yaxis_type = "Auto" # @param ["Auto", "linear", "log", "date"] {allow-input: true}
if yaxis_type == "Auto":
  yaxis_type = "-"

title = "Auto" # @param ["Auto"] {allow-input: true}
if title == "Auto":
  title = f"{selection_method.title()}: {selection}"
xaxis_title = "Auto" # @param ["Auto"] {allow-input: true}
if xaxis_title == "Auto":
  xaxis_title = "Page"
yaxis_title = "Auto" # @param ["Auto"] {allow-input: true}
if yaxis_title == "Auto":
  yaxis_title = yaxis_var


# Bar graph: article size vs language edition
# Different versions of x axis labels
plt = px.bar(
    data.sort(yaxis_var, descending=True).to_pandas(),
    x="selection_case",
    y=yaxis_var,
    template=template,
    color_discrete_sequence=px.colors.qualitative.Set2
)

plt.update_layout(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    # Rotate x-axis labels 45 degrees
    xaxis_tickangle=-45,
    yaxis_type=yaxis_type,
)

plt.show()

In [20]:
# @title Regression plot

template = "plotly_white"
xaxis_var = "length"  # @param ["length", "n_contributors", "n_revisions", "n_extlinks", "n_langlinks", "n_links", "n_linkshere", "n_categories", "pageviews_60d", "first_revision"] {allow-input: true}
yaxis_var = "n_links"  # @param ["length", "n_contributors", "n_revisions", "n_extlinks", "n_langlinks", "n_links", "n_linkshere", "n_categories", "pageviews_60d", "first_revision"] {allow-input: true}

title = "Auto"  # @param ["Auto"] {allow-input: true}
if title == "Auto":
    title = f"{selection_method.title()}: {selection}"
xaxis_title = "Auto"  # @param ["Auto"] {allow-input: true}
if xaxis_title == "Auto":
    xaxis_title = xaxis_var
yaxis_title = "Auto"  # @param ["Auto"] {allow-input: true}
if yaxis_title == "Auto":
    yaxis_title = yaxis_var


# Bar graph: article size vs language edition
# Different versions of x axis labels
plt = px.scatter(
    data.to_pandas(),
    x=xaxis_var,
    y=yaxis_var,
    template=template,
    color_discrete_sequence=px.colors.qualitative.Set2,
    # Add regression line
    trendline="ols",
    hover_name="selection_case",
)

plt.update_layout(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
)

plt.show()


## Plot Export

You may export your generated plot as a static image or interactive HTML.

### Formats
__HTML__: Interactive plot as shown above. May be viewed using a browser.  
__SVG__: Scalable vector graphic. Static image that may be resized without becoming blurry.  
__PNG__: Static raster image. May be pixelated or blurry due to Google Colab rendering.


### Options
`format`: Format to produce  
`filename`: Desired filename without extension  

__Non-interactive formats only (SVG/PNG)__  
`width` and `height`: Dimensions in pixels  
`scale`: Scale factor to use when exporting the figure. A scale factor larger than 1.0 will increase the image resolution with respect to the figure’s layout pixel dimensions.

In [9]:
# @title Options
format = "HTML"  # @param ["HTML", "SVG", "PNG"]
filename = "plot"  # @param {type:"string"}
width = 1200  # @param {type:"number"}
height = 600  # @param {type:"number"}
scale = 1  # @param {type:"number"}

plots_dir = "plots/"

output_file = "" + plots_dir + filename

if format == "HTML":
    output_file += ".html"
    plt.write_html(output_file)
elif format in ["PNG", "SVG"]:
    output_file += "." + format.lower()
    plt.write_image(
        output_file,
        scale=scale,
        width=width,
        height=height,
    )

files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>